In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e19/sample_submission.csv
/kaggle/input/playground-series-s3e19/train.csv
/kaggle/input/playground-series-s3e19/test.csv


In [ ]:
#installing pycaret
!pip install pycaret[regression]

In [ ]:
from pycaret.regression import *

In [14]:
#importing data
df = pd.read_csv('/kaggle/input/playground-series-s3e19/train.csv', parse_dates=['date'])
df['date'] = pd.to_datetime(df['date'])
test_df = pd.read_csv('/kaggle/input/playground-series-s3e19/test.csv', parse_dates=['date'])
test_df['date'] = pd.to_datetime(test_df['date'])

In [15]:
df.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [ ]:
#taking data from date column
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday

#features and dependent variable
X = df[['product', 'country', 'month', 'day', 'weekday', 'store']]
y = df['num_sold']

#encoding X
X = pd.get_dummies(X)

In [ ]:
#setting up pycaret
exp = setup(data=pd.concat([X, y], axis=1), target='num_sold', train_size=0.8, numeric_features=['month', 'day', 'weekday'])

#choosing top 3 models based on mae
top_models = compare_models(sort='MAE', n_select=3)

#creating model for these 3
models = [create_model(model) for model in top_models]

In [ ]:
# applying preprocessing to test data as well
test_df['month'] = test_df['date'].dt.month
test_df['day'] = test_df['date'].dt.day
test_df['weekday'] = test_df['date'].dt.weekday

X_test = test_df[['product', 'country', 'month', 'day', 'weekday', 'store']]

X_test = pd.get_dummies(X_test)

In [ ]:
#having same order for X_test and X
X_test = X_test[X.columns]

In [ ]:
#making predictions
y_preds = {}
for model in models:
    model_name = model.__class__.__name__
    predictions = predict_model(model, data=X_test)
    prediction_column = predictions.columns[-1] 
    y_preds[model_name] = predictions[prediction_column]

#combining predictions by taking mean from the dictionary
ensemble_pred = sum(y_preds.values()) / len(y_preds)

In [ ]:
# adjusting the ensemble predictions for inflation/growth as the test data is for next year
ensemble_pred_updated = ensemble_pred + 50

In [ ]:
output_df = pd.DataFrame({'id': test_df['id'], 'num_sold': ensemble_pred_updated})
output_df.to_csv('predictions.csv', index=False)